# Prima prova di neural network

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, Dataset
#from tqdm.notebook import tqdm, trange
from tqdm.auto import tqdm, trange
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from skimage.io import imread
from copy import deepcopy
import matplotlib.pyplot as plt
from sklearn import preprocessing
import pickle
import json
import sys
import time

prendo i dati, li leggo e li splitto

In [ ]:
train = None
with open('../data/images/data.json', 'r') as f:
    train = json.load(f)
labels = {}
train_img = []
y = []
for img_name, type in tqdm(train.items()):
    labels[type] = True
    y.append(type)
    image_path = '../data/' + str(img_name)
    img = imread(image_path, as_gray=True)
    # normalizing the pixel values
    img /= 255.0
    # converting the type of pixel to float 32
    img = img.astype('float32')
    train_img.append(img)


numLabel = len(labels)
# converting the list to numpy array
X = np.array(train_img)
y = np.array(y)

One hot encoding

In [ ]:
oneHot = False

In [ ]:
if oneHot:
    one = preprocessing.OneHotEncoder()
    pickle.dump(one, open('oneHotEncoder.pickle', 'wb'))
    y = one.fit_transform(y.reshape(-1,1)).toarray()
else:
    le = preprocessing.LabelEncoder()
    pickle.dump(le, open('labelEncoder.pickle', 'wb'))
    y = le.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.15)

#X_train = X_train.reshape(len(X), 16, 16)
X_train  = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)

# converting the target into torch format
y_train = y_train.astype(int)
y_train = torch.from_numpy(y_train)
y_test = y_test.astype(int)
y_test = torch.from_numpy(y_test)

# shape of training data
X_train.shape, y_train.shape

In [ ]:
X_train = X_train.unsqueeze(1)
X_test = X_test.unsqueeze(1)
y_test = y_test.type(torch.LongTensor)
y_train = y_train.type(torch.LongTensor)

In [ ]:
test = TensorDataset(X_test, y_test)
train = TensorDataset(X_train, y_train)

creo i batch, randomizzando i dati, con un dataloader

In [ ]:
batch_size = 50

# Create data loaders.
test = DataLoader(test, batch_size=batch_size, shuffle=True)
train = DataLoader(train, batch_size=batch_size, shuffle=True)

for X, y in test:
    print(f"Shape of X : {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

# creo la neural network

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self, numLabel):
        super(NeuralNetwork, self).__init__()
        #self.flatten = nn.Flatten()
        # take in input resistance, age, heater_r and heater_V
        self.linear_relu_stack = nn.Sequential(
            # Defining a 2D convolution layer
            nn.Conv2d(in_channels=1, out_channels=20, kernel_size=(5,5)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)),
            # second 2D convolution layer
            nn.Conv2d(in_channels=20, out_channels=50, kernel_size=(5,5)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)),
        )

        self.linear_layers = nn.Sequential(
            nn.Flatten(),
            # https://madebyollin.github.io/convnet-calculator/
            nn.Linear(50, numLabel),
            nn.LogSoftmax(dim=1)
        )


    def forward(self, x):
        #x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        logits = self.linear_layers(logits)
        return logits


In [ ]:
model = NeuralNetwork(numLabel=numLabel).to(device)
print(model)

In [ ]:
#loss_fn = nn.MSELoss()
#loss_fn = nn.L1Loss()
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-7, momentum=0.9)
LEARNING_RATE = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
#loss_fn = nn.CrossEntropyLoss()
loss_fn = nn.NLLLoss()

In [ ]:
train_losses = []
def trainf(dataloader, model, loss_fn, optimizer):
    global best_model
    best_loss = float("inf")
    size = len(dataloader.dataset)
    #model.double()
    model.train()
    for batch , (X, Y) in tqdm(enumerate(dataloader), leave=False, total=len(dataloader), position=1, file=sys.stdout):
        gas_loss = [([],[]) for _ in range(5)]
        X, Y = X.to(device), Y.to(device)
        loss = float("inf")
        optimizer.zero_grad()
        # Compute prediction error
        #output = torch.argmax(X, dim=1).float()
        pred = model(X)
        loss = loss_fn(pred,Y)
        loss.backward()
        # Backpropagation
        optimizer.step()

        if loss < best_loss:
            best_loss = loss
            best_model = deepcopy(model)

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            train_losses.append(loss)
            tqdm.write(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def testf(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        gas_avg_loss = [0 for _ in range(5)]
        for X, Y in dataloader:
            gas_loss = [([],[]) for _ in range(5)]
            X, Y = X.to(device), Y.to(device)
            pred = model(X)
            for i, (gas, y_lin) in enumerate(zip(pred, Y)):
                gas_loss[i][0].append(y_lin)
                gas_loss[i][1].append(gas)
            test_loss += loss_fn(pred, Y).item()
    test_loss /= num_batches * batch_size
    tqdm.write(f"Test Error: \n Avg loss: {test_loss:>8f} \n")


In [ ]:
epochs = 20
train_losses = []
val_losses = []
for t in trange(epochs, file=sys.stdout):
    #print(f"Epoch {t+1}\n-------------------------------")
    trainf(train, model, loss_fn, optimizer)
    testf(test, model, loss_fn)
#print("Done!")
torch.save(model, 'nn.torch')

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.show()